## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Extracción de reglas de pertenencia a grupos

##### Reglas de Asociación

---

En esta libreta se toman los resultados de la clusterización para realizar la extracción de reglas que permitan comprender de qué manera fueron conformados los grupos. 

---

### Carga de librerias a utilizar

In [ ]:
# Se importan las librerias a utilizar
import pandas as pd

# Para registro de toda la experimentación
import mlflow

# Para la extracción de reglas - puede requerir instalación. Ejecutar
# python -m pip install mlxtend

from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt

### Lectura del dataset generado previamente

In [ ]:
# Se hace la lectura del dataset del seminario y se obtiene su cabecera
datos = pd.read_csv('../../../data/final/resultados_kmeans_k2.csv', sep=';')
datos.head()

### Modificaciones del dataset para el descubrimiento de reglas

In [ ]:
# from mlxtend.frequent_patterns import apriori, association_rules

df_reglas = datos.copy()

# df_reglas['cluster'] = df_reglas['cluster'].astype('str')
df_reglas['duracion_credito'] = df_reglas['duracion_credito'].astype('str')
df_reglas['personas_a_cargo'] = df_reglas['personas_a_cargo'].astype('str')

valor_para_nan = 'NA'

# Importe solicitado

etiquetas_i_s = ['menor_5k', '5_a_15k', 'mayor_15k']
rangos_i_s = [0, 4, 15, 50]
importe_solicitado_N = pd.cut(df_reglas['importe_solicitado'], 
                                bins=rangos_i_s, 
                                labels=etiquetas_i_s,
                                right=False).cat.add_categories(valor_para_nan).fillna(valor_para_nan)

importe_solicitado_N.value_counts()

# Antiguedad del empleado

etiquetas_a_e = ['menor_5', '5_a_10', 'mayor_10']
rangos_a_e = [0, 4, 10, 50]
antiguedad_empleados_N = pd.cut(df_reglas['antiguedad_empleado'], 
                                bins=rangos_a_e, 
                                labels=etiquetas_a_e,
                                right=False).cat.add_categories(valor_para_nan).fillna(valor_para_nan)

antiguedad_empleados_N.value_counts()

# edad

etiquetas_e = ['menor_25', '25_a_30']
rangos_e = [0, 24, 50]
edad_N = pd.cut(df_reglas['edad'], 
                bins=rangos_e, 
                labels=etiquetas_e)

edad_N.value_counts()

# pct_ingreso

etiquetas_p_i = ['hasta_20', '20_a_40', '40_a_60', 'mayor_60']
rangos_p_i = [0, 0.19, 0.39, 0.60, 0.99]
pct_ingreso_N = pd.cut(df_reglas['pct_ingreso'], 
                                bins=rangos_p_i, 
                                labels=etiquetas_p_i)

pct_ingreso_N.value_counts()

# ingresos

etiquetas_i = ['hasta_20k', '20k_a_50k', '50k_a_100k', 'mayor_100k']
rangos_i = [0, 19999, 49999, 99999, 999999]
ingresos_N = pd.cut(df_reglas['ingresos'], 
                                bins=rangos_i, 
                                labels=etiquetas_i)

ingresos_N.value_counts()

# tasa_interes

etiquetas_t_i = ['hasta_7p', '7p_a_15p', '15p_a_20p', 'mayor_20p']
rangos_t_i = [0, 6.99, 14.99, 19.99, 100]
tasa_interes_N = pd.cut(df_reglas['tasa_interes'], 
                                bins=rangos_t_i, 
                                labels=etiquetas_t_i)

tasa_interes_N.value_counts()

# antiguedad_cliente

etiquetas_a_c = ['menor_2y', '2y_a_4y', 'mayor_4y']
rangos_a_c = [0, 24, 48, 100]
antiguedad_cliente_N = pd.cut(df_reglas['antiguedad_cliente'], 
                                bins=rangos_a_c, 
                                labels=etiquetas_a_c)

antiguedad_cliente_N.value_counts()

# limite_credito_tc

etiquetas_l_tc = ['menor_3k', '3k_a_5k', '5k_a_10k', 'mayor_10k']
rangos_l_tc = [0, 2999, 4999, 9999, 100000]
limite_credito_tc_N = pd.cut(df_reglas['limite_credito_tc'], 
                                bins=rangos_l_tc, 
                                labels=etiquetas_l_tc)

limite_credito_tc_N.value_counts()

# gastos_ult_12m

etiquetas_g_u12 = ['menor_1k', '2k_a_4k', '4k_a_6k', '6k_a_8k', '8k_a_10k', 'mayor_10k']
rangos_g_u12 = [0, 999, 3999, 5999, 7999, 9999, 100000]
gastos_ult_12m_N = pd.cut(df_reglas['gastos_ult_12m'], 
                                bins=rangos_g_u12, 
                                labels=etiquetas_g_u12)

gastos_ult_12m_N.value_counts()

# operaciones_ult_12m

etiquetas_o_u12 = ['menor_15', '15_a_30', '30_a_50', '50_a_75', '75_a_100', 'mayor_100']
rangos_o_u12 = [0, 14, 29, 49, 74, 99, 1000]
operaciones_ult_12m_N = pd.cut(df_reglas['operaciones_ult_12m'], 
                                bins=rangos_o_u12, 
                                labels=etiquetas_o_u12)

operaciones_ult_12m_N.value_counts()


cambios_estado_credito = {
    0: 'P',
    1 : 'C',
}

df_reglas['estado_credito'] = df_reglas['estado_credito'].map(cambios_estado_credito)

col_eliminar_final = [
              'edad',
              'importe_solicitado',
              'antiguedad_empleado',
              'antiguedad_cliente', 
              'regla_pct_ingresos_ok',
              'ingresos',
              'pct_ingreso', 
              'tasa_interes',
              'gastos_ult_12m', 
              'limite_credito_tc', 
              'operaciones_ult_12m',
              'id_cliente']

df_reglas.drop(col_eliminar_final, inplace=True, axis=1)
df_final = pd.concat([df_reglas, 
                      operaciones_ult_12m_N, 
                      gastos_ult_12m_N, 
                      limite_credito_tc_N, 
                      antiguedad_cliente_N, 
                      tasa_interes_N, 
                      ingresos_N, 
                      pct_ingreso_N, 
                      antiguedad_empleados_N,
                      importe_solicitado_N, 
                      edad_N], axis=1)
df_final.head(1)


In [ ]:
df_final.columns

In [ ]:
columnas_procesar = ['duracion_credito', 'estado_credito', 'personas_a_cargo',
       
       'operaciones_ult_12m', 'gastos_ult_12m', 'limite_credito_tc',
       'antiguedad_cliente', 'tasa_interes', 'ingresos', 'pct_ingreso',
       'antiguedad_empleado', 'importe_solicitado', 'edad']

data = pd.get_dummies(df_final, columns=columnas_procesar)
data.columns

In [ ]:
# Se genera el experimento en mlflow
exp_name = 'Experimentación extracción de reglas'
exp_id = mlflow.create_experiment(name=exp_name)

In [ ]:
# Group data by cluster
cluster_groups = data.groupby('cluster')

# Dictionary to store association rules for each cluster
cluster_rules = {}

# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Apriori - Soporte 0.2"):
    # Mine association rules for each cluster
    for cluster, data in cluster_groups:
        frequent_itemsets = apriori(data.drop(columns=['cluster']), min_support=0.2, use_colnames=True)
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
        cluster_rules[cluster] = rules

    # Log parameters
    mlflow.log_param("min_support", 0.2)
    mlflow.log_param("metric", "confidence")
    mlflow.log_param("min_threshold", 0.5)

    mlflow.log_metric("number_of_rules", len(rules))

    # Se finaliza el registro
    mlflow.end_run()


In [ ]:
# Print association rules for each cluster
for cluster, rules in cluster_rules.items():
    print(f"Cluster {cluster} Association Rules:")
    for idx, rule in rules.iterrows():
        antecedent = ', '.join(rule['antecedents'])
        consequent = ', '.join(rule['consequents'])
        support = round(rule['support'], 4)
        confidence = round(rule['confidence'], 4)
        lift = round(rule['lift'], 4)
        print(f"Rule {idx + 1}:")
        print(f"  Antecedent: {antecedent}")
        print(f"  Consequent: {consequent}")
        print(f"  Support: {support}")
        print(f"  Confidence: {confidence}")
        print(f"  Lift: {lift}\n")